# How to use Google calendar API?

How to import Google calendar API?


In [ ]:
var execSync = require('child_process').execSync;
try {
    require.resolve('googleapis');
    require.resolve('google-auth-library');
} catch (e) {
    execSync('npm install googleapis google-auth-library');
}
var google = require('googleapis');
var googleAuth = require('google-auth-library');

var importer = require('../Core');
var seleniumPath = __dirname + '/../Selenium/google authorize.ipynb';

// Authorize a client with the loaded credentials, then call the
  // Google Calendar API.
var oauth2Client;
var getOauthClient = (options = {}) => {
    return importer.importNotebook(seleniumPath)
    .then(r => r.authorize(['https://www.googleapis.com/auth/calendar']))
    .then(c => {
        var auth = new googleAuth();
        oauth2Client = new auth.OAuth2(
            c.clientId_, c.clientSecret_, c.redirectUri_);
        oauth2Client.credentials = c.credentials;
        return oauth2Client;
    })
    .then(auth => (options.auth = auth))
};
module.exports = getOauthClient;
getOauthClient;



How to list events?

In [ ]:
var importer = require('../Core');
var google = require('googleapis');

var calendar = google.calendar('v3');

var processResult = (err, response) => {
    if (err) {
      console.log('The API returned an error: ' + err);
      throw err;
    }
    resultEvents = []; // reset the array
    var events = response.items;
    if (events.length == 0) {
        console.log('No events found.');
    } else {
        for (var i = 0; i < events.length; i++) {
            var event = events[i];
            var start = event.start.dateTime || event.start.date;
            resultEvents[resultEvents.length] = {
                start: new Date(start), event: event};
        }
    }
    return resultEvents;
};


/**
 * Lists the next 10 events on the user's primary calendar.
 *
 * @param {google.auth.OAuth2} auth An authorized OAuth2 client.
 */
var resultEvents = [];
var getOauthClient, correctTimeLimits, correctCalendarId;
var listEvents = (options = {calendarId: 'primary'}) => {
    return importer.interpretAll([
        'import google calendar api',
        'correct dates time',
        'lookup calendar name'
    ])
    .then(r => {
        getOauthClient = r[0].runInNewContext();
        correctTimeLimits = r[1].runInNewContext();
        correctCalendarId = r[2].runInNewContext();
        if(typeof options.auth === 'undefined') {
            return getOauthClient(options);
        }
    })
    .then(() => correctTimeLimits(options))
    .then(() => correctCalendarId(options))
    .then(() => {
        if(typeof options['calendarId'] == 'undefined') {
            options['calendarId'] = 'primary';
        }
        var data = Object.assign({}, options, {
            maxResults: 100,
            singleEvents: true,
            orderBy: 'startTime'
        });
        return new Promise((resolve, reject) => {
            calendar.events.list(data, (err, response) => {
                if(err) reject(err);
                try {
                    var events = processResult(err, response);
                    resolve(events);
                } catch (e) {
                    reject(e);
                }
            });
        });
    });
};
module.exports = listEvents;
listEvents;



Run todays calendar events?



In [ ]:
var importer = require('../Core');
var google = require('googleapis');
var calendar = google.calendar('v3');

// test Google calendar API?
var getOauthClient, listEvents, correctCalendarId;
var runTodaysEvents = () => {
    var options = {}, commands = [];
    var eventId, isError;
    return importer.interpretAll([
        'list events',
        'import google calendar api',
        'lookup calendar name'
    ])
    .then(r => {
        listEvents = r[0].runInNewContext();
        getOauthClient = r[1].runInNewContext();
        correctCalendarId = r[2].runInNewContext();
        return getOauthClient(options);
    })
    .then(() => correctCalendarId(Object.assign(options, {
        calendarId: 'commands'
    })))
    .then(() => listEvents({
        auth: options.auth,
        calendarId: options.calendarId,
        timeMin: 'yesterday',
        timeMax: 'tomorrow',
    }))
    .then(events => {
        var names = events.map(e => e.event.summary);
        // filter processed
        commands = events
        .filter(e => e.event.summary.indexOf('Result:') == -1
                && names.indexOf('Result: ' + e.event.summary) == -1)
        
        // TODO: change this slice
        .slice(0, 1);
        if(commands.length == 0) throw new Error('No events!');
    })
    // create a new events to store the results
    .then(() => new Promise((resolve, reject) => {
        isError = false;
        calendar.events.insert(Object.assign({}, {
            calendarId: options.calendarId,
            auth: options.auth,
            resource: {
                start: commands[0].event.start,
                end: commands[0].event.end,
                summary: 'Result: ' + commands[0].event.summary,
                description: 'starting',
                colorId: 9
            }
        }), (err, response) => {
            if(err) reject(err);
            try {
                resolve(response);
            } catch (e) {
                reject(e);
            }
        });
    }))
    .then(event => (eventId = event.id))
    // process the command
    .then(() => importer.interpretAll(commands[0].event.summary))
    // TODO: add parameter input from event content
    // TODO: check for retry count from event content
    // TODO: add time limits for successful commands
    .then(r => r.runInNewContext()())
    .catch(e => {
        isError = true;
        return Object.getOwnPropertyNames(e).reduce((alt, key) => {
            alt[key] = e[key];
            return alt;
        }, {})
    })
    // update the calendar event with the results
    .then(r => new Promise((resolve, reject) => {
        console.log(r);
        if(typeof eventId == 'undefined') {
            return resolve(r);
        }
        calendar.events.patch({
            eventId: eventId,
            calendarId: options.calendarId,
            auth: options.auth,
            resource: {
                description: JSON.stringify(r, null, 4),
                colorId: isError ? 11 : 10
            }
        }, (err, response) => {
            if(err) reject(err);
            try {
                resolve(response);
            } catch (e) {
                reject(e);
            }
        });
    }))
};
module.exports = runTodaysEvents;
runTodaysEvents;

// echo "require('/Users/briancullinan/jupytangular2/Core').interpretAll('run todays calendar events').then(r=>r.runInNewContext()()).then(e=>{ console.log(e); process.exit(e); });" | node

// echo "require('/Users/briancullinan/jupytangular2/Core').interpretAll('scrape a bunch of facebook events').then(e=>{console.log(e); process.exit(e) });" | node


test calendar api graph Iga's cycle for as long as there are valid dates



In [ ]:
var importer = require('../Core');

$$.async();
var filterDistant, getOauthClient, listEvents, graphDates;
importer.interpretAll([
    'import google calendar api',
    'how to list events',
    'number of days between events',
    'display recurrence line graph'
])
.then(r => {
    getOauthClient = r[0].runInNewContext();
    listEvents = r[1].runInNewContext({getOauthClient});
    filterDistant = r[2].runInNewContext();
    graphDates = r[3].runInNewContext();
})
.then(() => listEvents({
    q: 'period',
    calendarId: 'p4685f6at8ccctenocjlqgnqss@group.calendar.google.com'
}))
.then(r => {
    $$.svg(graphDates(filterDistant(r)))
})
.catch(e => $$.sendError(e));




test calendar sum all hours worked on study sauce since september 2016



In [ ]:
var importer = require('../Core');

$$.async();
var sumEvents, getOauthClient, listEvents, ISODateString;
importer.interpretAll([
    'import google calendar api',
    'how to list events',
    'sum a list of events',
    'convert date to ISO'
])
.then(r => {
    getOauthClient = r[0].runInNewContext();
    listEvents = r[1].runInNewContext({getOauthClient});
    sumEvents = r[2].runInNewContext();
    ISODateString = r[3].runInNewContext();
})
.then(() => listEvents({
    timeMin: ISODateString(new Date('2016/09/22')),
    timeMax: ISODateString(new Date('2017/09/22')),
    q: 'study sauce'
}))
.then(r => {
    console.log(r);
    $$.sendResult(sumEvents(r))
})
.catch(e => $$.sendError(e));



Correct calendar dates for timeMax and timeMin?


In [ ]:
var importer = require('../Core');
var chrono = require('chrono-node');

var ISODateString;
var correctTimeLimits = (options) => {
    return importer.interpretAll([
        'convert date to ISO'
    ])
    .then(r => {
        ISODateString = r[0].runInNewContext();
        if(typeof options.timeMin !== 'undefined') {
            options.timeMin = ISODateString(
                chrono.parseDate(options.timeMin));
        }
        if(typeof options.timeMax !== 'undefined') {
            options.timeMax = ISODateString(
                chrono.parseDate(options.timeMax));
        }
        return options;
    });
};
module.exports = correctTimeLimits;
correctTimeLimits;



Lookup calendar id by name or id?



In [ ]:
var google = require('googleapis');
var calendar = google.calendar('v3');

var correctCalendarId = (options) => {
    options.calendarId = options.calendarId || 'primary';
    return new Promise((resolve, reject) => {
        calendar.calendarList.list({
            auth: options.auth,
        }, (err, response) => {
            if(err) reject(e);
            try {
                resolve(response);
            } catch (e) {
                reject(e);
            }
        });
    })
    .then(r => {
        var rexexp = new RegExp(options.calendarId, 'ig');
        var matches = r.items.filter(c => c.id == options.calendarId);
        if(matches.length == 0) {
            matches = r.items.filter(c => c.summary.match(rexexp));
        }
        console.log('Using calendar: ' + matches[0].summary
                    + ' - ' + matches[0].id);
        options.calendarId = matches[0].id;
        return options;
    })
};
module.exports = correctCalendarId;
correctCalendarId;



In [3]:
var importer = require('../Core');
var interpretAllPath = path.join(__dirname, '../Core/interpret all notebooks.ipynb');

var queries = 'scrape facebook events';

$$.async();
importer.importNotebook(interpretAllPath)
.then((f) => {
    interpret = f['interpret'];
    interpretObject = f['interpretObject'];
})
// use Promise.all to do all of your code searches up front
.then(r => typeof queries == 'string' 
      ? interpret(queries) 
      : Promise.all(queries.map(interpret)))
//.then(r => typeof queries == 'string' 
//      ? interpretObject([r[0]])
//      : interpretObject(r.map(r => r[0])))
.then(r => $$.sendResult(r))
.catch(e => $$.sendError(e))


Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]


[ 'facebook data.ipynb[6]',
  'facebook data.ipynb[2]',
  'facebook data.ipynb[7]',
  'facebook data.ipynb[4]',
  'facebook data.ipynb[1]',
  'google calendar.ipynb[2]',
  'linkedin messages.ipynb[1]',
  'linkedin messages.ipynb[4]',
  'levenshtein.ipynb[1]',
  'Angular components.ipynb[1]',
  'interpret questions.ipynb[1]' ]